In [63]:
%load_ext autoreload
%autoreload 2

import utils
import config
import pandas as pd
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
config.Grants.template

<function config.Grants.<lambda>(record)>

In [67]:
config.Grants.load(as_dataframe=True).head().grant_summary

0    Nanomaterials Optical Characterisation Facilit...
1    Advanced Geochemical Facility for Climate and ...
2    Efficient Coding for Distributed-input Distrib...
3    Fluid Dynamics of Circulation: Focus on the Ki...
4    Development of Novel Pesticidal Agents. We hav...
Name: grant_summary, dtype: object